In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('../../')

import psutil
import time

def monitor_memory(threshold=90, check_interval=1):
    """
    Monitors memory usage and kills the Jupyter process if it exceeds the threshold.
    
    Args:
        threshold (int): Percentage of memory usage before terminating the process.
        check_interval (int): Time (in seconds) between memory checks.
    """
    process = psutil.Process(os.getpid())  # Get current Jupyter process
    while True:
        mem_usage = psutil.virtual_memory().percent
        if mem_usage > threshold:
            print(f"⚠️ Memory usage exceeded {threshold}%! Killing process to prevent OOM...")
            process.terminate()  # Force kill Jupyter process
            break
        time.sleep(check_interval)  # Wait before next check

# Run the memory monitor in the background
import threading
memory_thread = threading.Thread(target=monitor_memory, args=(90, 5), daemon=True)
memory_thread.start()


In [2]:
from process_results import MetricsReader
import yaml
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
from pathlib import Path
pio.templates.default = "plotly_white"
pio.renderers.default = "browser"

In [6]:
plan = 'table2_r1'
results_dir = Path('results') / plan
plan_dir = Path('plans') / '250309_table2_r1'
cache_filepath = plan_dir / "results.msgpack"
LOAD_CACHE = True

In [4]:
SUCCESS_TIME_LIMIT = 120.0
ACCEPTABLE_MIN_DISTANCES_TO_TARGET = 0.10

In [7]:
import msgpack
import msgpack_numpy as mnp

results = {}

if LOAD_CACHE:
    def load_large_msgpack(filename):
        with open(filename, "rb") as f:
            unpacker = msgpack.Unpacker(f, raw=False, object_hook=mnp.decode)
            for key, value in unpacker:
                results[key] = value  # Load one entry at a time
    load_large_msgpack(cache_filepath)

else:
    filepaths = sorted(list(results_dir.glob('*.result')))
    results_str = {}
    for filepath in filepaths:
        try:
            reader = MetricsReader(
                filepath, 
                success_time_limit=SUCCESS_TIME_LIMIT, 
                acceptable_min_distances_to_target=ACCEPTABLE_MIN_DISTANCES_TO_TARGET
            )
            results[reader.info['label']] = reader
            results_str[reader.info['label']] = reader.format_stats()
        except Exception as e:
            print(f"Error reading {filepath}: {e}!")

    def save_large_msgpack(data, filename):
        with open(filename, "wb") as f:
            packer = msgpack.Packer(default=mnp.encode)
            for _, result in data.items():
                key = result.info['label']
                val = result.info
                f.write(packer.pack((key, val)))
    save_large_msgpack(results, cache_filepath)

## Analysis

In [36]:
scene = 'scene_'
use_cpu = 'use_cpu_'
pred_steps = 'pred_steps_'
pose = 'pose_'
DELIMITER = '-'


success_dfs = {}
mean_computation_times_dfs = {}
stddev_computation_times_dfs = {}
mean_durations_dfs = {}
stddev_durations_dfs = {}

# Extract unique values for each parameter
scene_ids = sorted(set([int(k.split(DELIMITER)[0].replace(scene, '')) for k in results.keys()]))
cpu_values = sorted(set([int(k.split(DELIMITER)[1].replace(use_cpu, '')) for k in results.keys()]))
pred_steps_values = sorted(set([int(k.split(DELIMITER)[2].replace(pred_steps, '')) for k in results.keys()]))

# For each scene, create a dataframe
for scene_id in scene_ids:
    # Initialize DataFrame with use_cpu as index and pred_steps as columns
    dfs = {}
    dfs['success'] = pd.DataFrame(index=cpu_values, columns=pred_steps_values)
    dfs['mean_computation_time'] = pd.DataFrame(index=cpu_values, columns=pred_steps_values)
    dfs['stddev_computation_time'] = pd.DataFrame(index=cpu_values, columns=pred_steps_values)
    dfs['mean_duration'] = pd.DataFrame(index=cpu_values, columns=pred_steps_values)
    dfs['stddev_duration'] = pd.DataFrame(index=cpu_values, columns=pred_steps_values)

    for df in dfs.values(): 
        df.index.name = 'use_cpu'
        df.columns.name = 'pred_steps'
     
    # Fill the dataframe with success counts
    for cpu_val in cpu_values:
        for pred_val in pred_steps_values:
            # Count successful poses (0-9) for this combination
            success_count = 0
            mean_computation_times = []
            stddev_computation_times = []
            durations = []

            for pose_id in range(10):  # poses 0-9
                key = f"{scene}{scene_id}{DELIMITER}{use_cpu}{cpu_val}{DELIMITER}{pred_steps}{pred_val}{DELIMITER}{pose}{pose_id}"
                if key in results: 
                    result = results[key]
                    if result.get('success', False):
                        if result.get('success') == 'True':
                            success_count += 1
                            if result.get('mean_planning_time', None) is not None:
                                mean_computation_times.append(result['mean_planning_time'])
                            if result.get('stddev_planning_time', None) is not None:
                                stddev_computation_times.append(result['stddev_planning_time'])
                            if result.get('elapsed_motion_runtime', None) is not None:
                                durations.append(result['elapsed_motion_runtime'])
                    
            dfs['success'].at[cpu_val, pred_val] = success_count
            dfs['mean_computation_time'].at[cpu_val, pred_val] = np.mean(mean_computation_times)
            dfs['stddev_computation_time'].at[cpu_val, pred_val] = np.mean(stddev_computation_times)
            dfs['mean_duration'].at[cpu_val, pred_val] = np.mean(durations)
            dfs['stddev_duration'].at[cpu_val, pred_val] = np.std(durations)
    
    success_dfs[scene_id] = dfs['success']
    mean_computation_times_dfs[scene_id] = dfs['mean_computation_time']
    stddev_computation_times_dfs[scene_id] = dfs['stddev_computation_time']
    mean_durations_dfs[scene_id] = dfs['mean_duration']
    stddev_durations_dfs[scene_id] = dfs['stddev_duration']

In [37]:
for scene_id, df in success_dfs.items():
    print(f"Scene {scene_id} Success Counts (out of 10 poses):")
    print(df)
    print()


Scene 0 Success Counts (out of 10 poses):
pred_steps  0  1  2
use_cpu            
0           7  8  7
1           8  7  8

Scene 1 Success Counts (out of 10 poses):
pred_steps  0  1  2
use_cpu            
0           9  9  9
1           9  9  9

Scene 2 Success Counts (out of 10 poses):
pred_steps   0   1   2
use_cpu               
0           10  10  10
1           10  10  10

Scene 3 Success Counts (out of 10 poses):
pred_steps  0  1  2
use_cpu            
0           9  9  9
1           9  9  9



In [38]:
# Define the metrics we want to include
metrics = ['success', 'mean_computation_time', 'stddev_computation_time', 'mean_duration', 'stddev_duration']

# Create a dictionary to store all data for the hierarchical DataFrame
all_data = {}

# Iterate through each scene and CPU value
for scene_id in scene_ids:
    for cpu_val in cpu_values:
        # For each prediction step value
        for pred_val in pred_steps_values:
            # For each metric
            for metric in metrics:
                # Create a tuple key for the hierarchical index
                row_idx = (scene_id, cpu_val)
                col_idx = (pred_val, metric)
                
                # Get the value from the appropriate DataFrame
                if metric == 'success':
                    value = success_dfs[scene_id].at[cpu_val, pred_val]
                elif metric == 'mean_computation_time':
                    value = mean_computation_times_dfs[scene_id].at[cpu_val, pred_val]
                elif metric == 'stddev_computation_time':
                    value = stddev_computation_times_dfs[scene_id].at[cpu_val, pred_val]
                elif metric == 'mean_duration':
                    value = mean_durations_dfs[scene_id].at[cpu_val, pred_val]
                elif metric == 'stddev_duration':
                    value = stddev_durations_dfs[scene_id].at[cpu_val, pred_val]
                
                # Store the value in our data dictionary
                all_data[(row_idx, col_idx)] = value

# Create MultiIndex for rows and columns
row_index = pd.MultiIndex.from_product([scene_ids, cpu_values], names=['scene_id', 'use_cpu'])
col_index = pd.MultiIndex.from_product([pred_steps_values, metrics], names=['pred_steps', 'metric'])

# Create the hierarchical DataFrame
hierarchical_df = pd.DataFrame(index=row_index, columns=col_index)

# Fill the DataFrame with values
for (row_idx, col_idx), value in all_data.items():
    hierarchical_df.at[row_idx, col_idx] = value

# Display the hierarchical DataFrame
print("Hierarchical DataFrame with all metrics:")
hierarchical_df

Hierarchical DataFrame with all metrics:


pred_steps             0                                                \
metric           success mean_computation_time stddev_computation_time   
scene_id use_cpu                                                         
0        0             7              0.077529                0.003632   
         1             8              0.264908                0.050523   
1        0             9              0.079503                0.003144   
         1             9              0.079594                0.003291   
2        0            10              0.079175                0.003112   
         1            10              0.115293                0.029633   
3        0             9              0.078268                0.003058   
         1             9              0.162119                0.018853   

pred_steps                                           1                        \
metric           mean_duration stddev_duration success mean_computation_time   
scene_id use_cpu                                                               
0        0           33.624219        3.100748       8              0.153714   
         1           30.666469        2.727907       7              0.522292   
1        0           44.899662        7.560583       9              0.158009   
         1           47.237081        6.646904       9              0.158376   
2        0           39.707815       12.917523      10              0.156805   
         1           37.802962       11.313552      10               0.22779   
3        0           27.984866        5.348951       9              0.155303   
         1           26.704413        3.805611       9              0.322221   

pred_steps                                                              \
metric           stddev_computation_time mean_duration stddev_duration   
scene_id use_cpu                                                         
0        0                      0.004552     37.115193        7.007561   
         1                      0.104469     31.828679        2.988685   
1        0                      0.005039      49.01698        9.890633   
         1                      0.005403     48.962432       10.207566   
2        0                      0.005599     40.708855       17.003012   
         1                      0.060152     41.259261       11.453228   
3        0                       0.00469     39.893608       21.175588   
         1                      0.034399     28.820284         3.84641   

pred_steps             2                                                \
metric           success mean_computation_time stddev_computation_time   
scene_id use_cpu                                                         
0        0             7              0.230569                0.005598   
         1             8              0.785745                0.152964   
1        0             9              0.236221                0.005904   
         1             9              0.236873                0.006316   
2        0            10              0.235239                 0.00581   
         1            10              0.339429                0.090271   
3        0             9              0.233009                0.005717   
         1             9              0.482346                0.048437   

pred_steps                                      
metric           mean_duration stddev_duration  
scene_id use_cpu                                
0        0           39.273052        3.478498  
         1           32.797093        2.834169  
1        0           46.067478       13.317345  
         1           52.246093       12.179336  
2        0           42.161359       10.835715  
         1           42.508544       12.002621  
3        0           35.440093        4.790465  
         1            29.43072        4.012876